### 株価DBウェブサイトのHTML上の株価データをスクレイピング

- 参考URL(株価データベースからクローリング&スクレイピング)<br>
https://manareki.com/stock_python_scraping

- 参考URL(スクレイピングで株価データを取得する)<br>
https://non-dimension.com/kabuka-scraping/

- 参考URL(Pythonで作る株価予測SlackBot)<br>
https://qiita.com/ogi-iii/items/cc9a7bc45f7b15001440

- 参考URL(QuantXで機械学習使って株価予測してみた)<br>
https://qiita.com/sarubobo29/items/b94ccb6c1305c157fafb<br>
https://factory.quantx.io/developer/3d4fd86b53a948b3827f689caf931297



In [1]:
# ライブラリのインポート
import os
import ssl
import csv
import glob
import time
import urllib
import random
import requests

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver

from sklearn.preprocessing import Imputer

In [2]:
# csv保存ファイルを設定
path = "/Users/KojiroT/github/Stock_Data/"

# Line証券の取扱銘柄一覧を取得
line_df = pd.read_csv(path + 'line.csv', header=0)
line_df.head()

# codeをリストに格納
line_code = line_df['Code']
print(line_code.values.tolist()[:5])

# 銘柄をリストに格納
line_name = line_df['Name']
print(line_name.values.tolist()[:5])

[1605, 1801, 1928, 2127, 2229]
['国際石油開発帝石  ', '大成建設  ', '積水ハウス  ', '日本Ｍ＆Ａセンター  ', 'カルビー  ']


In [3]:
# コードと銘柄を設定
code_values = line_code
name_values = line_name
# 辞書に格納
stock_dict = {"code": code_values, "name": name_values}
# pandasのdfに変換、表示
df = pd.DataFrame.from_dict(stock_dict)
df.head()

,code,name
0,1605,国際石油開発帝石
1,1801,大成建設
2,1928,積水ハウス
3,2127,日本Ｍ＆Ａセンター
4,2229,カルビー


In [60]:
def get_dfs(stock_number):
    dfs = []
    year = [2017,2018,2019] #2017〜2019年までの株価データを取得
    for y in year:
        try:
            print(y)
            url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
            soup = BeautifulSoup(requests.get(url).content,'html.parser')
            tag_tr = soup.find_all('tr')
            head = [h.text for h in tag_tr[0].find_all('th')]
            data = []
            for i in range(1,len(tag_tr)):
                data.append([d.text for d in tag_tr[i].find_all('td')])
            df = pd.DataFrame(data, columns = head)

            col = ['始値','高値','安値','終値','出来高','終値調整']
            for c in col:
                df[c] = df[c].astype(float)
            df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]
            dfs.append(df)
        except IndexError:
            print('No data')
    return dfs

# pandas整形処理
def concatenate(dfs):
    data = pd.concat(dfs,axis=0)
    data = data.reset_index(drop=True)
    col = ['始値','高値','安値','終値','出来高','終値調整']
    for c in col:
        data[c] = data[c].astype(float)
    return data

#複数のデータフレームをcsvで保存
for i in range(len(df)):
    k = df.loc[i,'code']
    v = df.loc[i,'name']
    print(k,v)
    dfs = get_dfs(k)
    data = concatenate(dfs) 
    data.to_csv(path + '{}-{}.csv'.format(k,v))

1605 国際石油開発帝石  
2017
2018
2019
1801 大成建設  
2017
2018
2019
1928 積水ハウス  
2017
2018
2019
2127 日本Ｍ＆Ａセンター  
2017
2018
2019
2229 カルビー  
2017
2018
2019
2269 明治ＨＤ  
2017
2018
2019
2317 システナ  
2017
2018
2019
2326 デジタルアーツ  
2017
2018
2019
2371 カカクコム  
2017
2018
2019
2413 エムスリー  
2017
2018
2019
2432 ディー⋅エヌ⋅エー  
2017
2018
2019
2502 アサヒグループＨＤ  
2017
2018
2019
2587 サントリー食品インターナショナル  
2017
2018
2019
2593 伊藤園  
2017
2018
2019
2702 日本マクドナルドＨＤ  
2017
2018
2019
2811 カゴメ  
2017
2018
2019
2897 日清食品ＨＤ  
2017
2018
2019
2914 日本たばこ産業  
2017
2018
2019
3064 ＭｏｎｏｔａＲＯ  
2017
2018
2019
3088 マツモトキヨシＨＤ  
2017
2018
2019
3092 ＺＯＺＯ  
2017
2018
2019
3197 すかいらーくＨＤ  
2017
2018
2019
3382 セブン＆アイ⋅ＨＤ  
2017
2018
2019
3563 スシローグローバルＨＤ  
2017
2018
2019
3861 王子ＨＤ  
2017
2018
2019
3923 ラクス  
2017
2018
2019
3990 ＵＵＵＭ  
2017
2018
2019
4063 信越化学工業  
2017
2018
2019
4324 電通  
2017
2018
2019
4344 ソースネクスト  
2017
2018
2019
4384 ラクスル  
2017
No data
2018
2019
4385 メルカリ  
2017
No data
2018
2019
4452 花王  
2017
2018
2019
4502 武田薬品工業  
2017
201

### データ前処理

#### 取得データの整形

1. csvを取得
2. 特徴量の追加
  - pandasで行・列の差分・変化率を取得するdiffを使う
      - diff()で前の行との差分を算出する  Change.diff()
      - diff(-1)で1行後の列の差分を算出する  Change.diff(-1)
  - pct_change()で(A - B) / Bの変化率を算出する<br>
    https://note.nkmk.me/python-pandas-diff-pct-change/
  - diffメソッドで行・列の差分を計算<br>
    https://algorithm.joho.info/programming/python/pandas-diff/
  - データフレームに特徴量を追加する
3. 欠損値処理
  - 欠損値を前の値で置き換える
  - データフレームの欠損値を確認
4. 教師データの生成
5. データ要約の表示


次のフェーズでcsv上書き

In [4]:
# csvの取得 & 特徴量の追加 & 欠損値処理
# path = "/Users/KojiroT/github/Stock_Data/" 上で設定済

# 取得済みcsvをロード
df = pd.read_csv(path + '1801-大成建設  .csv', header=0)

# 不要なカラムを除去
df = df.drop("Unnamed: 0", axis=1) 

#  終値のみを一旦リストにしてpandasにする
change_list = df['終値']
change_df = pd.DataFrame(change_list)

# 前日比（率） : Change ＝ （終値 － 1日前の終値） ÷ 1日前の終値 × 100
Beforeday_Change = change_df.pct_change()
Beforeday_Change = Beforeday_Change.rename(columns={'終値': '前日比(率)'})

# 翌日比（率） : Nextday_Change＝ （終値 － 1日後の終値） ÷ 1日後の終値 × 100
Nextday_Change = change_df.pct_change(-1)
Nextday_Change = Nextday_Change.rename(columns={'終値': '翌日比(率)'})

# データフレームに前日比と翌日比を結合させる
df = pd.concat([df, Beforeday_Change, Nextday_Change], axis=1)

# 欠損値処理(欠損値を前の値で置き換えた後、後の値で置き換える)
df = df.fillna(method='ffill').fillna(method='bfill')

# 欠損値の確認
# df.isnull().sum()

# データの要約を表示
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 9 columns):
日付        671 non-null object
始値        671 non-null float64
高値        671 non-null float64
安値        671 non-null float64
終値        671 non-null float64
出来高       671 non-null float64
終値調整      671 non-null float64
前日比(率)    671 non-null float64
翌日比(率)    671 non-null float64
dtypes: float64(8), object(1)
memory usage: 47.3+ KB


In [5]:
# 教師データの生成

# label(教師データ)の作成
label = []

# 教師データのチューニングパラメータ
period = 1 # 何日以内
percent = 0.001 # 何パーセント予測するか

# 正解データの生成(終値のchange_listをここで使用する)
for i in range(len(change_list)-period):
    for j in range(1, period+1):
        rate=0
        rate=(change_list[i+j]-change_list[i])/change_list[i]
        if rate>=percent:
            label.append(1)
            break
        elif rate<=-percent:
            label.append(-1)
            break
        elif j==period:
            label.append(0)

# 最後のperiod個は値を入れられてないので代わりに0を入れとく(必須)
for i in range(period):
    label.append(0)

# 教師データをpandasに格納する
print(label[:5])
answer = pd.DataFrame(label, columns=['answer'])

# dfに教師データを結合する
df = pd.concat([df, answer], axis=1)
df.head()

[0, 1, -1, 1, -1]


,日付,始値,高値,安値,終値,出来高,終値調整,前日比(率),翌日比(率),answer
0,2017-01-04,836.0,837.0,826.0,835.0,5077000.0,4175.0,0.000000,0.000000,0
1,2017-01-05,835.0,836.0,828.0,835.0,5317000.0,4175.0,0.000000,-0.009490,1
2,2017-01-06,836.0,844.0,835.0,843.0,5559000.0,4215.0,0.009581,0.004768,-1
3,2017-01-10,843.0,843.0,835.0,839.0,4864000.0,4195.0,-0.004745,-0.008274,1
4,2017-01-11,842.0,846.0,835.0,846.0,4467000.0,4230.0,0.008343,0.020507,-1


In [6]:
# データ要約の表示
print(df.info())
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 10 columns):
日付        671 non-null object
始値        671 non-null float64
高値        671 non-null float64
安値        671 non-null float64
終値        671 non-null float64
出来高       671 non-null float64
終値調整      671 non-null float64
前日比(率)    671 non-null float64
翌日比(率)    671 non-null float64
answer    671 non-null int64
dtypes: float64(8), int64(1), object(1)
memory usage: 52.5+ KB
None


,日付,始値,高値,安値,終値,出来高,終値調整,前日比(率),翌日比(率),answer
666,2019-09-26,4315.0,4345.0,4260.0,4270.0,1421300.0,4270.0,0.010651,0.010651,-1
667,2019-09-27,4205.0,4235.0,4135.0,4225.0,1423200.0,4225.0,-0.010539,0.009558,-1
668,2019-09-30,4225.0,4230.0,4145.0,4185.0,785000.0,4185.0,-0.009467,0.006010,-1
669,2019-10-01,4200.0,4230.0,4145.0,4160.0,715800.0,4160.0,-0.005974,0.007264,-1
670,2019-10-02,4130.0,4155.0,4105.0,4130.0,787100.0,4130.0,-0.007212,0.007264,0


### CSVの上書き

- 読み込みモードの指定 mode<br>
  デフォルトでmode="w"になっており、同じ名前のファイルがあったとしても内容が上書きされることになっている<br>
  上書きせずデータをそのまま付け足す形で使いたいときはmode="a"にすれば上書きではなく書き足すことが可能<br>
  ex) `df.to_csv("name.csv", mode="a", header=False)`

In [8]:
df.to_csv(path + "Stock_Data.csv", header=True)

### 参考コード

株価チャートを画像データとして学習させる深層学習による株式市場の予測
https://ai-scholar.tech/treatise/stock-price-ai-107/

In [1]:
# https://manareki.com/stock_python_scraping
'''
# 変数とpathを設定
code = "9984"
year = "2019"
path = "/Users/KojiroT/github/Stock_Data/"    # csv保存先を指定

# Urlinbモジュール(websサイト接続)
url = "https://kabuoji3.com/stock/"+code+"/"+year+"/" 

ssl._create_default_https_context = ssl._create_unverified_context    # sslエラー回避
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})   # request処理をfirefox指定
html = urllib.request.urlopen(req).read() 

# BeautifulSoupモジュール
soup = BeautifulSoup(html, "html.parser")
time.sleep(random.randrange(3,7))

stockdata= soup.find_all("td")    # tableのtd要素を抽出
stockdata = [s.contents[0] for s in stockdata]
stockdata = list(zip(*[iter(stockdata)]*7))    # <td></td>を取り除いた後、７列のリストに変換

# pandasモジュール
df = pd.DataFrame(stockdata,)
df.columns=['date','open','high','low','close','valume','adjclose']
df.to_csv(path+code+".csv")
'''

'\n# 変数とpathを設定\ncode = "9984"\nyear = "2019"\npath = "/Users/KojiroT/github/Stock_Data/"    # csv保存先を指定\n\n# Urlinbモジュール(websサイト接続)\nurl = "https://kabuoji3.com/stock/"+code+"/"+year+"/" \n\nssl._create_default_https_context = ssl._create_unverified_context    # sslエラー回避\nreq = urllib.request.Request(url, headers={\'User-Agent\': \'Mozilla/5.0\'})   # request処理をfirefox指定\nhtml = urllib.request.urlopen(req).read() \n\n# BeautifulSoupモジュール\nsoup = BeautifulSoup(html, "html.parser")\ntime.sleep(random.randrange(3,7))\n\nstockdata= soup.find_all("td")    # tableのtd要素を抽出\nstockdata = [s.contents[0] for s in stockdata]\nstockdata = list(zip(*[iter(stockdata)]*7))    # <td></td>を取り除いた後、７列のリストに変換\n\n# pandasモジュール\ndf = pd.DataFrame(stockdata,)\ndf.columns=[\'date\',\'open\',\'high\',\'low\',\'close\',\'valume\',\'adjclose\']\ndf.to_csv(path+code+".csv")\n'

In [2]:
# https://non-dimension.com/kabuka-scraping/
'''
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

def get_dfs(stock_number):
    dfs = []
    year = [2017,2018,2019] #2017〜2019年までの株価データを取得
    for y in year:
        try:
            print(y)
            url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
            soup = BeautifulSoup(requests.get(url).content,'html.parser')
            tag_tr = soup.find_all('tr')
            head = [h.text for h in tag_tr[0].find_all('th')]
            data = []
            for i in range(1,len(tag_tr)):
                data.append([d.text for d in tag_tr[i].find_all('td')])
            df = pd.DataFrame(data, columns = head)

            col = ['始値','高値','安値','終値','出来高','終値調整']
            for c in col:
                df[c] = df[c].astype(float)
            df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]
            dfs.append(df)
        except IndexError:
            print('No data')
    return dfs

def concatenate(dfs):
    data = pd.concat(dfs,axis=0)
    data = data.reset_index(drop=True)
    col = ['始値','高値','安値','終値','出来高','終値調整']
    for c in col:
        data[c] = data[c].astype(float)
    return data

#複数のデータフレームをcsvで保存
for i in range(len(df)):
    k = df.loc[i,'code']
    v = df.loc[i,'name']
    print(k,v)
    dfs = get_dfs(k)
    data = concatenate(dfs) 
    data.to_csv('{}-{}.csv'.format(k,v))
'''

"\nfrom bs4 import BeautifulSoup\nimport pandas as pd\nimport requests\nfrom datetime import datetime\n\ndef get_dfs(stock_number):\n    dfs = []\n    year = [2017,2018,2019] #2017〜2019年までの株価データを取得\n    for y in year:\n        try:\n            print(y)\n            url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)\n            soup = BeautifulSoup(requests.get(url).content,'html.parser')\n            tag_tr = soup.find_all('tr')\n            head = [h.text for h in tag_tr[0].find_all('th')]\n            data = []\n            for i in range(1,len(tag_tr)):\n                data.append([d.text for d in tag_tr[i].find_all('td')])\n            df = pd.DataFrame(data, columns = head)\n\n            col = ['始値','高値','安値','終値','出来高','終値調整']\n            for c in col:\n                df[c] = df[c].astype(float)\n            df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]\n            dfs.append(df)\n        except IndexError:\n            print('No data')\n    

In [27]:
'''
# seleniumのDLフォルダを指定
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "/Users/KojiroT/github/Stock_Data/"}
chromeOptions.add_experimental_option("prefs",prefs)

#Chrome diriverのパス
browser = webdriver.Chrome(executable_path='/Users/KojiroT/github/Stock_Data/chromedriver', chrome_options=chromeOptions)

#予測したい株価の前日比データを集める / stock scraping (comparison with yesterday)
browser.get("https://kabuoji3.com/stock/{}/".format(4689))
stockTable=browser.find_element_by_class_name("table_wrap")
stockLine=stockTable.find_elements_by_tag_name("tr")
targetStockComparisons=[]
for i in range(2,152):
    targetStockPriceAfter=stockLine[i-1].find_elements_by_tag_name("td")
    targetStockPriceBefore=stockLine[i].find_elements_by_tag_name("td")
    targetStockComparison=float(targetStockPriceAfter[6].text)-float(targetStockPriceBefore[6].text)
    targetStockComparisons.append(targetStockComparison)
targetStockPriceAfter=stockLine[151].find_elements_by_tag_name("td")
targetStockPriceBefore=stockLine[153].find_elements_by_tag_name("td")
targetStockComparison=float(targetStockPriceAfter[6].text)-float(targetStockPriceBefore[6].text)
targetStockComparisons.append(targetStockComparison)
for i in range(154,302):
    targetStockPriceAfter=stockLine[i-1].find_elements_by_tag_name("td")
    targetStockPriceBefore=stockLine[i].find_elements_by_tag_name("td")
    targetStockComparison=float(targetStockPriceAfter[6].text)-float(targetStockPriceBefore[6].text)
    targetStockComparisons.append(targetStockComparison)
df=pd.DataFrame(targetStockComparisons)
df.columns=["yahoo：前日比"]
df

#add table
stockPriceTable=pd.concat([df_date,ETFTable],axis=1)
stockPriceTable=pd.concat([stockPriceTable,df],axis=1)
stockPriceTable.head()

#予測したい株価の前日比データを集める / stock scraping (comparison with yesterday)

#prepare for making target values
df_next=df.copy()
df_next.columns=["(株)野村総合研究所：翌日比"]

#date scraping for target values
browser.get("https://kabuoji3.com/stock/{}/".format(4307))
stockTable=browser.find_element_by_class_name("table_wrap")
stockLine=stockTable.find_elements_by_tag_name("tr")
dates=[]
for i in range(2,152):
    stockDate=stockLine[i].find_elements_by_tag_name("td")
    stockDate=stockDate[0].text
    dates.append(stockDate)
for i in range(153,302):
    stockDate=stockLine[i].find_elements_by_tag_name("td")
    stockDate=stockDate[0].text
    dates.append(stockDate)
df_date2=pd.DataFrame()
df_date2["date"]=dates

#making target values table
df_next=pd.concat([df_date2,df_next],axis=1)
df_next.index=df_date2["date"]

#prepare for complete table
table=stockPriceTable[1:299].copy()
table.index=table["date"]

#making complete table
table["(株)野村総合研究所：翌日比"]=df_next["(株)野村総合研究所：翌日比"]
table.tail()
'''

'\n# seleniumのDLフォルダを指定\nchromeOptions = webdriver.ChromeOptions()\nprefs = {"download.default_directory" : "/Users/KojiroT/github/Stock_Data/"}\nchromeOptions.add_experimental_option("prefs",prefs)\n\n#Chrome diriverのパス\nbrowser = webdriver.Chrome(executable_path=\'/Users/KojiroT/github/Stock_Data/chromedriver\', chrome_options=chromeOptions)\n\n#予測したい株価の前日比データを集める / stock scraping (comparison with yesterday)\nbrowser.get("https://kabuoji3.com/stock/{}/".format(4689))\nstockTable=browser.find_element_by_class_name("table_wrap")\nstockLine=stockTable.find_elements_by_tag_name("tr")\ntargetStockComparisons=[]\nfor i in range(2,152):\n    targetStockPriceAfter=stockLine[i-1].find_elements_by_tag_name("td")\n    targetStockPriceBefore=stockLine[i].find_elements_by_tag_name("td")\n    targetStockComparison=float(targetStockPriceAfter[6].text)-float(targetStockPriceBefore[6].text)\n    targetStockComparisons.append(targetStockComparison)\ntargetStockPriceAfter=stockLine[151].find_elements_b

In [39]:
'''
# 欠損値処理(平均値置換)
# 不要なカラムを除去
KB = kesson.drop("日付", axis=1) 
KB.head()

# インピュータークラスのインスタンス化と（列平均の）学習
imp = Imputer()
imp.fit(KB)

# 学習済みImputerの適用：各列の欠損値の置換
kesson_columns = kesson.columns.values
kesson_fin = pd.DataFrame(imp.transform(kesson), columns=kesson_columns)

# 結果表示
kesson_fin.head()
'''

'\n# 欠損値処理(平均値置換)\n# 不要なカラムを除去\nKB = kesson.drop("日付", axis=1) \nKB.head()\n\n# インピュータークラスのインスタンス化と（列平均の）学習\nimp = Imputer()\nimp.fit(KB)\n\n# 学習済みImputerの適用：各列の欠損値の置換\nkesson_columns = kesson.columns.values\nkesson_fin = pd.DataFrame(imp.transform(kesson), columns=kesson_columns)\n\n# 結果表示\nkesson_fin.head()\n'

In [24]:
'''
# http://www.stockdog.work/entry/2017/02/09/211119
# ここで学習用のデータを作ります。
# 学習データの作り方は、x日のデータ（ETFの前日比）を説明変数に、x+1日の騰落を目的変数に入れます。
# x+1日の株価が上昇していれば、+1を入れ、下落していれば-1を入れます。

x_train = []
y_train = []
for s in range(0, len(df_train) - 1):
    print("x_train : ", df_train["Date"].iloc[s])
    print("y_train : ", df_train["Date"].iloc[s + 1])
    print("")
    x_train.append(df_train[xlist].iloc[s])

    if df_train["Close"].iloc[s + 1] > df_train["Close"].iloc[s]:
        y_train.append(1)
    else:
    y_train.append(-1)

#print(x_train)
#print(y_train)
'''

'\n# http://www.stockdog.work/entry/2017/02/09/211119\n# ここで学習用のデータを作ります。\n# 学習データの作り方は、x日のデータ（ETFの前日比）を説明変数に、x+1日の騰落を目的変数に入れます。\n# x+1日の株価が上昇していれば、+1を入れ、下落していれば-1を入れます。\n\nx_train = []\ny_train = []\nfor s in range(0, len(df_train) - 1):\n    print("x_train : ", df_train["Date"].iloc[s])\n    print("y_train : ", df_train["Date"].iloc[s + 1])\n    print("")\n    x_train.append(df_train[xlist].iloc[s])\n\n    if df_train["Close"].iloc[s + 1] > df_train["Close"].iloc[s]:\n        y_train.append(1)\n    else:\n    y_train.append(-1)\n\n#print(x_train)\n#print(y_train)\n'

In [25]:
'''
# 機械学習で未来を予測する - scikit-learn の決定木で未来の株価を予測
# https://qiita.com/ynakayama/items/6a472e5ebbe9365186bd

def train_data(arr):
    train_X = []
    train_y = []
    # 30 日間のデータを学習、 1 日ずつ後ろにずらしていく
    for i in np.arange(-30, -15):
        s = i + 14 # 14 日間の変化を素性にする
        feature = arr.ix[i:s]
        if feature[-1] < arr[s]: # その翌日、株価は上がったか？
            train_y.append(1) # YES なら 1 を
        else:
            train_y.append(0) # NO なら 0 を
        train_X.append(feature.values)
    # 上げ下げの結果と教師データのセットを返す
    return np.array(train_X), np.array(train_y)
'''

'\n# 機械学習で未来を予測する - scikit-learn の決定木で未来の株価を予測\n# https://qiita.com/ynakayama/items/6a472e5ebbe9365186bd\n\ndef train_data(arr):\n    train_X = []\n    train_y = []\n    # 30 日間のデータを学習、 1 日ずつ後ろにずらしていく\n    for i in np.arange(-30, -15):\n        s = i + 14 # 14 日間の変化を素性にする\n        feature = arr.ix[i:s]\n        if feature[-1] < arr[s]: # その翌日、株価は上がったか？\n            train_y.append(1) # YES なら 1 を\n        else:\n            train_y.append(0) # NO なら 0 を\n        train_X.append(feature.values)\n    # 上げ下げの結果と教師データのセットを返す\n    return np.array(train_X), np.array(train_y)\n'